<a href="https://colab.research.google.com/github/aykhangaffarov/Generative_Models/blob/initial-commit/Pre_Upsampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
print(tf.__version__)

# Environment prep for Colab
 - download git repo 
 - install required pip packages

In [ ]:
!git clone https://github.com/aykhangaffarov/Generative_Models # this is for loading git with correct brach

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/My\ Drive/Results
MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
IMG_DIR = '/content/Generative_Models/data/.CBSD68' #change .qbert here as well

%cd /content/Generative_Models/
!pip -q install -r /content/Generative_Models/requirements.txt
!pip install colorlog==4.0.2
%cd /content/

In [ ]:
#!mv Generative_Models/data/.CBSD68/CBSD68.tar.bz Generative_Models/data/

# Settings

In [ ]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
os.chdir('Generative_Models')
print(os.getcwd())

# Unzip

In [ ]:
!tar -xvf data/CBSD68_low2high_LMDB.tar.bz 

In [ ]:
!mv data/.CBSD68_low2high_LMDB/ data/SR

In [ ]:
!ls data/SR

# **Loading** Dataset

In [ ]:
from training.generators.from_lmdb.lmdb_image_generator import get_generators

In [ ]:
!pip install colorlog

In [ ]:
dataset_name='CBSD68_low2high'
lmdb_dir = 'data/SR'

In [ ]:
image_size_original=(481, 321, 3)
scale = 2
image_size = list(map(lambda x: x//scale , image_size_original[:-1])) + [image_size_original[-1]]
image_size = (*image_size,)
batch_size = 16
latents_dim = 150
intermediate_dim = 50

In [ ]:
from LMDB_Datasets.transformation.lmdb_transformer import LmdbTransformer

In [ ]:
#inputs_shape = list(map(lambda x: x//2 , image_size[:-1])) + [image_size[-1]]
#inputs_shape = (*inputs_shape, )

In [ ]:
#shrink_fn = lambda image: tf.image.resize(image, inputs_shape[:-1])
#enlarge_fn = lambda image: tf.image.resize(image, image_size[:-1])

In [ ]:
def generator_reshaper(generator): 
    while True:
        batch = next(generator)
        yield batch['image_target'], batch['image_source']

In [ ]:
train_generator, val_generator = get_generators(
        lmdb_dir=lmdb_dir,
        batch_size=batch_size,
        episode_len=None,
        episode_shift=None,
        return_itr=True
)

In [ ]:
data = next(train_generator)
print(data.keys())

In [ ]:
import numpy as np

**Problem is when we return data from generator it didn't seems like float type.**

In [ ]:
np.frombuffer(data['image_source'][0],dtype=np.float32)

In [ ]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_generator, 
    output_types= (tf.float32, tf.float32),
)
val_ds = tf.data.Dataset.from_generator(
    lambda: val_generator, 
    output_types= (tf.float32, tf.float32),
)

In [ ]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [ ]:
data.keys()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(np.frombuffer(data['image_source'][2], dtype=np.float32))
trace_image=data['image_source'][2]

Models Layers definition

# input is half

In [ ]:
image_size, inputs_shape

In [ ]:
image_size

In [ ]:
import numpy as np
from collections.abc import Iterable
if isinstance(image_size, Iterable):
    _outputs_shape = np.prod(image_size)
_outputs_shape    

# Model's Layers definition

In [ ]:
kernel_size=3
stride = 2

In [ ]:
c = list(map(lambda x: x// (stride*stride), image_size[:-1]))
c = (*c, intermediate_dim)
c

In [ ]:
enc_lays = [
    tf.keras.layers.UpSampling2D(size=(2, 2)),
    tf.keras.layers.UpSampling2D(size=(2, 2)),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=np.product(c), activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=c),
    tf.keras.layers.Conv2DTranspose(filters=intermediate_dim, kernel_size=kernel_size, strides=(stride, stride), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=intermediate_dim, kernel_size=kernel_size, strides=(stride, stride), padding="SAME", activation='relu'),
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=kernel_size, strides=(1, 1), padding="SAME")
]

# Model definition

In [ ]:
model_name = dataset_name+'_Conv_Pre_Upsampling_LMDB'
#windows
#experiments_dir='..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..'+sep_local+'experiments'+sep_local + model_name

#linux 
experiments_dir=os.getcwd()+ sep_local  +'experiments'+sep_local + model_name

In [ ]:
variables_params = \
[
    {
        'name': 'inference',  #'upsampler',
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', #'constructive', 
        'inputs_shape':latents_dim,
        'outputs_shape':image_size,
        'layers':dec_lays
    }
]

In [ ]:
from os.path import abspath
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(experiments_dir, 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)
absolute = abspath(experiments_dir)
print("Recording_dir",absolute)
print("Current working dir",os.getcwd())

In [ ]:
from training.autoencoding_basic.transformative.AE import autoencoder as AE

In [ ]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=None,#to restore trained model, set filepath=_restore
    episode_len=1
    )

In [ ]:
image_size, inputs_shape

In [ ]:
ae.compile(metrics=None)

#ae.compile(metrics=create_metrics())
#ae.compile()

# Callbacks

In [ ]:
# added for linux warning suppression
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)


from training.callbacks.trace_image_reconstruction import trace_reconstruction
from training.callbacks.save_model import ModelSaver

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=12, 
    verbose=1, 
    restore_best_weights=False
)

In [ ]:
ms = ModelSaver(filepath=_restore)

In [ ]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

In [ ]:
image_reconstuction_dir = os.path.join(experiments_dir, 'image_reconstuction_dir')
create_if_not_exist(image_reconstuction_dir)
absolute = abspath(image_reconstuction_dir)
print("image_reconstuction_dir",absolute)

In [ ]:
image = shrink_fn(np.asarray(trace_image)).numpy()/255.0
img_reconst = trace_reconstruction(filepath=image_reconstuction_dir, image=image, gen_freq=5)

# Model Training

In [ ]:
ae.fit(
    x=train_generator,
    input_kw=None,
    steps_per_epoch=int(1e4),
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, img_reconst],
    workers=-1,
    use_multiprocessing=True,
    validation_data=val_generator,
    validation_steps=int(1e4)
    )

FORK

In [ ]:
!git checkout -b initial-commit

In [ ]:
!git add .


In [ ]:
!git commit -m 'initial commit'

In [ ]:
!git push origin initial-commit